In [117]:
!pip install icalendar
!pip3 install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [118]:
import json
from bs4 import BeautifulSoup 
import urllib.request
import requests

iCal parsing into rdf graph


In [119]:
import icalendar
import rdflib
import regex as re

# Load the ical file
with open('/content/DSCCal.ics', 'rb') as f:
    calendar = icalendar.Calendar.from_ical(f.read())


for component in calendar.walk():
    rdf_file=""
    g = rdflib.Graph()

    Schema = rdflib.Namespace("https://schema.org/")
    g.bind('schema',Schema, override=True)
    id =''
    emse_base = "https://territoire.emse.fr/kg/emse/fayol/"
    if component.name == "VEVENT":
        
        
        # This is an event. Extract the relevant information.
        id = component.get('uid')
        location = component.get('location')
        startdate = component.get('dtstart').dt
        enddate = component.get('dtend').dt
        if 'EMSE' in location:
          organizer = 'Ecole des Mines de Saint-Etienne'
          location = location.replace("EMSE S",'').replace("EMSE Espace Fauriel S",'').replace(".",'')
          location = emse_base + location[0] + 'ET/' + location
        else:
          organizer = 'Universite Jean Monnet'


        event_description = re.sub(r"\n", " ", component.get('description')).split('(Exporté')
        description = event_description[0][2:-1]

        title = component.get('summary')
        Course = rdflib.Namespace("https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/"+id+"/")
        g.bind('course',Course)
        courseURI = "https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/"+id+"/"

        
        # Add this information to the RDF graph
        g.add((rdflib.URIRef(courseURI), rdflib.RDF.type, Schema.CourseInstance))
        g.add((rdflib.URIRef(courseURI), Schema.title, rdflib.Literal(title)))
        if organizer == 'Ecole des Mines de Saint-Etienne':
          g.add((rdflib.URIRef(courseURI), Schema.location, rdflib.URIRef(location)))
        else :
          g.add((rdflib.URIRef(courseURI), Schema.location, rdflib.Literal(location)))
        g.add((rdflib.URIRef(courseURI), Schema.organizer, rdflib.Literal(organizer)))
        g.add((rdflib.URIRef(courseURI), Schema.startDate, rdflib.Literal(startdate)))
        g.add((rdflib.URIRef(courseURI), Schema.endDate, rdflib.Literal(enddate)))
        g.add((rdflib.URIRef(courseURI), Schema.description, rdflib.Literal(description)))
        with open('/content/Courses/'+id+'.ttl', 'w') as j:
          j.write(g.serialize(format='turtle'))

Getting data of external events from the website Alentoor

In [120]:
import json
import rdflib
from bs4 import BeautifulSoup as bs
response = requests.get('https://www.alentoor.fr/saint-etienne/agenda')
soup = BeautifulSoup(response.content, 'html.parser')
events_jsonld = soup.find_all('script', type='application/ld+json')
events = []
# Afficher le contenu de chaque script ld+json
for event_jsonld in events_jsonld:
    event = {}
    event_jsonld = event_jsonld.text
    events.append(json.loads(event_jsonld))

g = rdflib.Graph()

Schema = rdflib.Namespace("https://schema.org/")
g.bind('schema',Schema, override=True)

Event = rdflib.Namespace("https://territoire.emse.fr/ldp/")
g.bind('event',Event)
eventURI="https://territoire.emse.fr/ldp/"

for event in events:
        g = rdflib.Graph()

        Schema = rdflib.Namespace("https://schema.org/")
        g.bind('schema',Schema, override=True)
        id=event['@id'].split("https://www.alentoor.fr/agenda/")[1]
        Ext_Event = rdflib.Namespace("https://territoire.emse.fr/ldp/Fadl-NaitBachir/events/"+id+"/")
        g.bind('ext_event',Ext_Event)
        Ext_EventURI = "https://territoire.emse.fr/ldp/Fadl-NaitBachir/events/"+id+"/"
        id=event['@id'].split("https://www.alentoor.fr/agenda/")[1]
        g.add((rdflib.URIRef(Ext_EventURI), rdflib.RDF.type, Schema.Event))
        g.add((rdflib.URIRef(Ext_EventURI), Schema.title, rdflib.Literal(event['name'])))
        g.add((rdflib.URIRef(Ext_EventURI), Schema.location, rdflib.Literal(event['location']['name'])))
        g.add((rdflib.URIRef(Ext_EventURI), Schema.startDate, rdflib.Literal(event['startDate'], datatype=rdflib.XSD.date)))
        g.add((rdflib.URIRef(Ext_EventURI), Schema.endDate, rdflib.Literal(event['endDate'],datatype=rdflib.XSD.date)))
        g.add((rdflib.URIRef(Ext_EventURI), Schema.description, rdflib.Literal(event['description'])))
        with open('/content/Events/'+id+'.ttl', 'w') as j:
          j.write(g.serialize(format='turtle'))

Create the first container which contains the container of courses and the container of events

In [121]:
import requests
from base64 import b64encode


server_url = 'https://territoire.emse.fr/ldp/'

username = 'ldpuser'
password = 'LinkedDataIsGreat'

# Encode the username and password for the request
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')


# The file path of the RDF graph in Turtle format
rdf_file ="/content/test0.ttl"

# Read the RDF graph from the file
with open(rdf_file, 'r') as f:
    rdf_data = f.read()


slug = 'Fadl_NaitBachir'

headers = {
    'Content-Type': 'text/turtle',
    'Authorization': f'Basic {encoded_credentials}',
    'Slug': slug,
}
response = requests.post(server_url, headers=headers, data=rdf_data)

# Check the response status code to see if the request was successful
if response.status_code == 201:
    print('Resource successfully created')
else:
    print(f'Error: {response.status_code}')

Resource successfully created


In [122]:
import requests
from base64 import b64encode


server_url = 'https://territoire.emse.fr/ldp/Fadl-NaitBachir/'

username = 'ldpuser'
password = 'LinkedDataIsGreat'

# Encode the username and password for the request
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')


# The file path of the RDF graph in Turtle format
rdf_file = '/content/coursesContainer.ttl'

# Read the RDF graph from the file
with open(rdf_file, 'r') as f:
    rdf_data = f.read()


slug = 'Fadl_NaitBachir'

headers = {
    'Content-Type': 'text/turtle',
    'Authorization': f'Basic {encoded_credentials}',
    'Slug': slug,
}
response = requests.post(server_url, headers=headers, data=rdf_data)

# Check the response status code to see if the request was successful
if response.status_code == 201:
    print('Resource successfully created')
else:
    print(f'Error: {response.status_code}')

Resource successfully created


In [123]:
import requests
from base64 import b64encode


server_url = 'https://territoire.emse.fr/ldp/Fadl-NaitBachir/'

username = 'ldpuser'
password = 'LinkedDataIsGreat'

# Encode the username and password for the request
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')


# The file path of the RDF graph in Turtle format
rdf_file = '/content/eventsContainer.ttl'

# Read the RDF graph from the file
with open(rdf_file, 'r') as f:
    rdf_data = f.read()

# Optional: specify a name for the created resource
slug = 'Fadl_NaitBachir'

headers = {
    'Content-Type': 'text/turtle',
    'Authorization': f'Basic {encoded_credentials}',
    'Slug': slug,
}
response = requests.post(server_url, headers=headers, data=rdf_data)

# Check the response status code to see if the request was successful
if response.status_code == 201:
    print('Resource successfully created')
else:
    print(f'Error: {response.status_code}')

Resource successfully created


Publishing rdf graphs on the containers of courses and events

In [124]:
import os

server_url="https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/"
folder_path = '/content/Courses/'

for file_name in os.listdir(folder_path):
    rdf_file = folder_path +file_name
    # Read the RDF graph from the file
    with open(rdf_file, 'r') as f:
        rdf_data = f.read()

    # Optional: specify a name for the created resource
    slug = 'Fadl_NaitBachir'

    headers = {
        'Content-Type': 'text/turtle',
        'Authorization': f'Basic {encoded_credentials}',
        'Slug': slug,
    }
    response = requests.post(server_url, headers=headers, data=rdf_data)

    # Check the response status code to see if the request was successful
    if response.status_code == 201:
        print('Resource successfully created')
    else:
        print(f'Error: {response.status_code}')

Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource s

In [125]:
server_url = 'https://territoire.emse.fr/ldp/Fadl-NaitBachir/events/'

# Define the username and password for the request
username = 'ldpuser'
password = 'LinkedDataIsGreat'

# Encode the username and password for the request
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')


import os

folder_path = '/content/Events/'

for file_name in os.listdir(folder_path):
    rdf_file = folder_path +file_name
    # Read the RDF graph from the file
    with open(rdf_file, 'r') as f:
        rdf_data = f.read()

    # Optional: specify a name for the created resource
    slug = 'Fadl_NaitBachir'

    headers = {
        'Content-Type': 'text/turtle',
        'Authorization': f'Basic {encoded_credentials}',
        'Slug': slug,
    }
    response = requests.post(server_url, headers=headers, data=rdf_data.encode('utf-8'))

    # Check the response status code to see if the request was successful
    if response.status_code == 201:
        print('Resource successfully created')
    else:
        print(f'Error: {response.status_code}')

Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created
Resource successfully created


SPARQL Queries that we used

In [126]:
#return all the events
import requests
from base64 import b64encode

url =  'https://territoire.emse.fr/ldp/FadlNaitBachir/events/'

query = f"""PREFIX ldp: <http://www.w3.org/ns/ldp#>
PREFIX sh: <https://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?resource ?start_date ?end_date ?desc ?tit ?loc
WHERE {{
    <{url}> ldp:member ?resource .
    ?resource sh:startDate ?start_date;
         sh:endDate ?end_date;
         sh:description ?desc;
         sh:title ?tit;
         sh:location ?loc.
}}
"""

username = 'ldpuser'
password = 'LinkedDataIsGreat'

# Encode the username and password for the request
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')

headers = {
    "Accept": "application/sparql-results+json",
    "Content-Type": "application/sparql-query",
    'Authorization': f'Basic {encoded_credentials}'
}

response = requests.post(url, headers=headers, data = query)
extevents = {}

events = response.json()
for i in range (len(events['results']['bindings'])):
  data = events['results']['bindings'][i]
  extevents[data["resource"]["value"].\
          split('https://territoire.emse.fr/ldp/FadlNaitBachir/events/')[1]] = \
          {'start_date': data['start_date']['value'], 'description':data['desc']['value'],\
          'end_date':data['start_date']['value'],'location':data['loc']['value'],\
          'title':data['tit']['value']}
extevents

{'4014406/': {'start_date': '2023-01-13',
  'description': 'Pierre Dumoussaud, jeune trentenaire, fait partie de la nouvelle génération des chefs d’orchestre en vue. Il dirige le temps d’un concert l’Orchestre Symphonique Saint-Étienne Loire, pour une soirée c',
  'end_date': '2023-01-13',
  'location': 'Grand Théâtre Massenet',
  'title': "Opéra de Saint-Etienne - Magie de l'orchestre français"},
 '1862692/': {'start_date': '2023-01-13',
  'description': 'Marché de produits manufacturés et alimentaires.',
  'end_date': '2023-01-13',
  'location': 'Rue Edouard Lalo',
  'title': 'Marché de Montreynaud'},
 '2542863/': {'start_date': '2023-01-13',
  'description': 'Marché de produits manufacturés et alimentaires.',
  'end_date': '2023-01-13',
  'location': 'Place de la République',
  'title': 'Marché de Côte-Chaude'},
 '3490125/': {'start_date': '2023-01-13',
  'description': 'Marché de produits manufacturés et alimentaires.',
  'end_date': '2023-01-13',
  'location': 'Place Aristide Bria

In [127]:
#return all the courses
url =  'https://territoire.emse.fr/ldp/FadlNaitBachir/courses/'

query = f"""PREFIX ldp: <http://www.w3.org/ns/ldp#>
PREFIX sh: <https://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?resource ?start_date ?end_date ?desc ?tit ?loc
WHERE {{
    <{url}> ldp:member ?resource .
    ?resource sh:startDate ?start_date;
         sh:endDate ?end_date;
         sh:description ?desc;
         sh:title ?tit;
         sh:location ?loc.
}}
"""

username = 'ldpuser'
password = 'LinkedDataIsGreat'

# Encode the username and password for the request
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')


response = requests.post(url, headers=headers, data = query)
courses = {}
events = response.json()
for i in range (len(events['results']['bindings'])):
    data = events['results']['bindings'][i]
    courses[data["resource"]["value"].\
          split('https://territoire.emse.fr/ldp/FadlNaitBachir/courses/')[1]] = \
          {'start_date': data['start_date']['value'], 'description':data['desc']['value'],\
          'end_date':data['start_date']['value'],'location':data['loc']['value'],\
          'title':data['tit']['value']}
courses

{'ADE60323032322d3230323353542d455449454e4e452d32313037362d312d30/': {'start_date': '2022-11-09T12:30:00+00:00',
  'description': 'CC M2 PE/AIMA M2 DSC VERRON SYLVAIN',
  'end_date': '2022-11-09T12:30:00+00:00',
  'location': 'L202 r�serv�e IAE S1 21-22 - Carnot - Bat. des forges',
  'title': 'CM/TD Economie Gestion'},
 'ADE60323032322d3230323353542d455449454e4e452d353738352d312d30/': {'start_date': 'rnths',
  'description': 'rtgsxb',
  'end_date': 'rnths',
  'location': 'sgfhsr',
  'title': 'azeetyer'},
 'ADE60323032322d3230323353542d455449454e4e452d32323537312d382d30/': {'start_date': '13',
  'description': 'Soutenance',
  'end_date': '13',
  'location': 'jean monnet',
  'title': 'Semantic web exam'},
 'ADE60323032322d3230323353542d455449454e4e452d343037302d372d30/': {'start_date': '2022-10-27T12:00:00+00:00',
  'description': 'M2 DSC Alternance M2 DSC GERY MATHIAS',
  'end_date': '2022-10-27T12:00:00+00:00',
  'location': 'L215 (34) (Manufacture)',
  'title': "CM/TD Recherche d'info

In [128]:
#upcoming courses
#return all the courses
url =  'https://territoire.emse.fr/ldp/FadlNaitBachir/courses/'

query = f"""PREFIX ldp: <http://www.w3.org/ns/ldp#>
PREFIX sh: <https://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?resource ?start_date ?end_date ?desc ?tit ?loc
WHERE {{
    <{url}> ldp:member ?resource .
    ?resource sh:startDate ?start_date;
         sh:endDate ?end_date;
         sh:description ?desc;
         sh:title ?tit;
         sh:location ?loc.
    FILTER(?start_date > NOW())
}}
"""

username = 'ldpuser'
password = 'LinkedDataIsGreat'

# Encode the username and password for the request
credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')


response = requests.post(url, headers=headers, data = query)
courses = {}
events = response.json()
for i in range (len(events['results']['bindings'])):
    data = events['results']['bindings'][i]
    courses[data["resource"]["value"].\
          split('https://territoire.emse.fr/ldp/FadlNaitBachir/courses/')[1]] = \
          {'start_date': data['start_date']['value'], 'description':data['desc']['value'],\
          'end_date':data['start_date']['value'],'location':data['loc']['value'],\
          'title':data['tit']['value']}
courses

{'ADE60323032322d3230323353542d455449454e4e452d343037302d382d30/': {'start_date': '2023-01-26T13:00:00+00:00',
  'description': 'M2 DSC Alternance M2 DSC GERY MATHIAS',
  'end_date': '2023-01-26T13:00:00+00:00',
  'location': 'L203 a+b (64) (Manufacture)',
  'title': "CM/TD Recherche d'information"},
 'ADE60323032322d3230323353542d455449454e4e452d313136342d302d30/': {'start_date': '2023-01-24T13:00:00+00:00',
  'description': 'M2 DSC M2 MLDM M2 DSC Alternance M2 CPE-MLDM2 (0) JEUDY BAPTISTE',
  'end_date': '2023-01-24T13:00:00+00:00',
  'location': 'L203 a+b (64) (Manufacture)',
  'title': 'EXAM Data Mining for Big Data (+TSE-DSC)'},
 'ADE60323032322d3230323353542d455449454e4e452d34383636302d302d30/': {'start_date': '2023-01-19T07:30:00+00:00',
  'description': 'M2 AIMA (4+4TSE) M2 PE (6+1TSE) M2 DSC M2 DSC Alternance Master 2 Chimie Sciences des Mat�riaux BOST CHRYSTEL',
  'end_date': '2023-01-19T07:30:00+00:00',
  'location': 'Amphi D201 (308)',
  'title': 'TOEIC M�tare'}}

SHACL Validation 

In [129]:
!pip install pyshacl
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Testing the validation of an rdf graph of an event

In [130]:
from pyshacl import validate
from rdflib import Graph

shacl = Graph().parse("./shacl.ttl", 'turtle')
g = Graph().parse("/content/Events/2542865.ttl", 'turtle')
conforms,m,d= validate(g, shacl_graph=shacl)
print(conforms)

True


Other functionalities we worked on :

Delete

In [131]:
import requests
from base64 import b64encode
url = "https://territoire.emse.fr/ldp/Fadl-NaitBachir/events/2542856/"

username = 'ldpuser'
password = 'LinkedDataIsGreat'

credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')
slug = 'Fadl_NaitBachir'

headers = {
    'Content-Type': 'text/turtle',
    'Authorization': f'Basic {encoded_credentials}',
    'Slug': slug,
}
response = requests.delete(url, headers=headers)
response.raise_for_status()
print(response.text)

Get

In [132]:
url = "https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/"

# récupération de la ressource existante
username = 'ldpuser'
password = 'LinkedDataIsGreat'

credentials = f'{username}:{password}'.encode('utf-8')
encoded_credentials = b64encode(credentials).decode('utf-8')
slug = 'Fadl_NaitBachir'

headers = {
    'Content-Type': 'text/turtle',
    'Authorization': f'Basic {encoded_credentials}',
    'Slug': slug,
}
response = requests.get(url, headers=headers)
response.text

'\n<https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/>\n  a <http://www.w3.org/ns/ldp#BasicContainer>, <http://www.w3.org/ns/ldp#Container>,\n    <http://www.w3.org/ns/ldp#RDFSource>, <http://www.w3.org/ns/ldp#Resource>, <https://carbonldp.com/ns/v1/platform#Document>,\n    <https://schema.org/CourseInstance>;\n  <http://www.w3.org/ns/ldp#hasMemberRelation> <http://www.w3.org/ns/ldp#member>;\n  <http://www.w3.org/ns/ldp#insertedContentRelation> <http://www.w3.org/ns/ldp#MemberSubject>;\n  <http://www.w3.org/ns/ldp#membershipResource> <https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/>;\n  <https://carbonldp.com/ns/v1/platform#created> "2023-01-13T21:25:23.593Z"^^<http://www.w3.org/2001/XMLSchema#dateTime>;\n  <https://carbonldp.com/ns/v1/platform#modified> "2023-01-13T21:25:23.593Z"^^<http://www.w3.org/2001/XMLSchema#dateTime>;\n  <https://schema.o

Update:

In [133]:
#we try here to update the event that we got in  the previous code
data = response.text
g = rdflib.Graph()
g.parse(data=data, format='turtle')

predicate=['title','location','startDate','endDate','description']
predicates = [rdflib.URIRef('https://schema.org/'+item) for item in predicate] 
old_triples=[]
for x in predicates:  
  result = g.triples((None, x, None))
  for s, p, o in result:
      old_triple = (s, p, o)
      old_triples.append(old_triple)
new_triples=[]
new_values=['Semantic Web','EMSE',"2022-11-24T16:00:00+00:00","2022-11-24T13:00:00+00:00",'Soutenance']
for w in old_triples:
  new_triples.append((w[0], w[1], rdflib.Literal(new_values[old_triples.index(w)])))
  print(new_triples[old_triples.index(w)])

(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/'), rdflib.term.URIRef('https://schema.org/title'), rdflib.term.Literal('Semantic Web'))
(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/'), rdflib.term.URIRef('https://schema.org/location'), rdflib.term.Literal('EMSE'))
(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/'), rdflib.term.URIRef('https://schema.org/startDate'), rdflib.term.Literal('2022-11-24T16:00:00+00:00'))
(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/'), rdflib.term.URIRef('https://schema.org/endDate'), rdflib.term.Literal('2022-11-24T13:00:00+00:00'))
(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-

In [134]:
#new we gonna remove the old triples and add the new ones, as well as removing the triple that are not related to the list of predicates we determined
for w in old_triples:
  g.remove(w)
  g.add(new_triples[old_triples.index(w)])
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/ns/ldp#hasMemberRelation'), rdflib.term.URIRef('http://www.w3.org/ns/ldp#member')))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('https://carbonldp.com/ns/v1/platform#modified'), None))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('https://carbonldp.com/ns/v1/platform#created'), None))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/ns/ldp#Container')))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/ns/ldp#RDFSource')))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/ns/ldp#BasicContainer')))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/ns/ldp#Resource')))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/ns/ldp#hasMemberRelation'), rdflib.term.URIRef('http://www.w3.org/ns/ldp#member')))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/ns/ldp#membershipResource'), rdflib.term.URIRef(url)))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/ns/ldp#insertedContentRelation'), rdflib.term.URIRef('http://www.w3.org/ns/ldp#MemberSubject')))
g.remove((rdflib.term.URIRef(url), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://carbonldp.com/ns/v1/platform#Document')))

<Graph identifier=N38ea4950ccea4920b41dc4a3c9ab8ce0 (<class 'rdflib.graph.Graph'>)>

In [135]:
for s in g:
  print(s)

(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('https://schema.org/CourseInstance'))
(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/'), rdflib.term.URIRef('https://schema.org/organizer'), rdflib.term.Literal('Universite Jean Monnet'))
(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/'), rdflib.term.URIRef('https://schema.org/description'), rdflib.term.Literal('Soutenance'))
(rdflib.term.URIRef('https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/'), rdflib.term.URIRef('https://schema.org/location'), rdflib.term.Literal('EMSE'))
(rdflib.term.URIRef('https:/

In [136]:
new_resource = g.serialize(format='turtle')
new_resource

'@prefix ns1: <https://schema.org/> .\n\n<https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/ADE60323032322d3230323353542d455449454e4e452d32383435322d302d31/> a ns1:CourseInstance ;\n    ns1:description "Soutenance" ;\n    ns1:endDate "2022-11-24T13:00:00+00:00" ;\n    ns1:location "EMSE" ;\n    ns1:organizer "Universite Jean Monnet" ;\n    ns1:startDate "2022-11-24T16:00:00+00:00" ;\n    ns1:title "Semantic Web" .\n\n'

In [137]:
#we delete the resource which presents the old version and we create using post the updated version of this resource 
new_resource = g.serialize(format='turtle')
response = requests.delete(url, headers=headers)
url_cont="https://territoire.emse.fr/ldp/Fadl-NaitBachir/courses/"
response = requests.post(url_cont, headers=headers, data=new_resource)
if response.status_code == 201:
    print('Resource successfully created')
else:
    print(f'Error: {response.status_code}')

Resource successfully created
